In [1]:
# python detect_mask_video.py

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [2]:
# to capture the image 
def detect_and_predict_mask(frame,faceNet,maskNet):
    # to get the dimension of the frame and to construct the blob
    (h,w)= frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame,1.0,(300,300),
                                 (104.0,177.0,123.0))
    
    #pass the blob through model and detect the face using faceNet
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    #initialize the list of face and the corresponding locatins and the list of predicitons from our face mask model
    
    faces =[]
    locs =[]
    preds =[]
    
    # loop over detections
    for i in range(0,detections.shape[2]):
        # extract the probability
        confidence = detections[0,0,i,2]
        
        #filter out weak detections by ensuring the confidence/probability value
        # (i.e) the confidence value is greater than the min conf value
        
        if confidence>args['confidence']:
            #compute the x and y coordinates of bounding box 
            
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")
            
            #ensure the box falls within the face dim
            (startX,startY) =(max(0,startX),max(0,startY))
            (endX,endY) = (min(w-1,endX),min(h-1,endY))
            face = frame[startY:endY,startX:endX]

            # Debugging statements
            print(f"Detection {i}: confidence={confidence}")
            print(f"Bounding box: startX={startX}, startY={startY}, endX={endX}, endY={endY}")
            print(f"Face shape: {face.shape}")

            if face.size == 0:
                print("Empty face image detected, skipping...")
                continue
            
            #extract the face ROI, convert it to BGR to RGB channel ordering, resize it to 224x224, and preprocess it
            # face = frame[startY:endY,startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(224,224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            # add the face and bounding box to the respective list
            faces.append(face)
            locs.append((startX,startY,endX,endY))
            
    # to give a condition to predict atleast one face was detected
    if len(faces)>0:
# for faster inference we'll make batch predictions on *all*
# faces at the same time rather than one-by-one predictions
# in the above `for` loop
        faces = np.array(faces,dtype ='float32')
        preds = maskNet.predict(faces,batch_size = 32)
        
    return (locs,preds)
        


In [4]:
# construct the argument parser and parse the arguments
# Construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--face", type=str, default="face_detector", help="path to face detector model directory")
ap.add_argument("-m", "--model", type=str, default="mask_detector.h5", help="path to trained face mask detector model")
ap.add_argument("-c", "--confidence", type=float, default=0.5, help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

# Construct the absolute paths to the model files
project_root = os.path.abspath(os.path.dirname(r"C:\HOPE\Deep Learning\Face Mask Dectection\face_detector"))
face_detector_directory = os.path.join(project_root, 'face_detector')
prototxtPath = os.path.join(face_detector_directory, "deploy.prototxt")
weightsPath = os.path.join(face_detector_directory, "res10_300x300_ssd_iter_140000.caffemodel")

print(f"Prototxt path: {prototxtPath}")
print(f"Weights path: {weightsPath}")

if not os.path.exists(prototxtPath):
    print(f"Error: {prototxtPath} not found.")
if not os.path.exists(weightsPath):
    print(f"Error: {weightsPath} not found.")

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)



Prototxt path: C:\HOPE\Deep Learning\Face Mask Dectection\face_detector\deploy.prototxt
Weights path: C:\HOPE\Deep Learning\Face Mask Dectection\face_detector\res10_300x300_ssd_iter_140000.caffemodel


In [5]:
# Load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model(args["model"])


[INFO] loading face mask detector model...


In [6]:
# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)



[INFO] starting video stream...


In [9]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
        # display the label and bounding box rectangle on the output
        # frame
        if(label=="Mask"):    
            cv2.putText(frame,"Mask: You are allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        elif(label=="No Mask"):
            lab="No Mask: You are not allowed"
            cv2.putText(frame, lab, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

vs.release()

Detection 0: confidence=0.9850894212722778
Bounding box: startX=192, startY=121, endX=268, endY=223
Face shape: (102, 76, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Detection 0: confidence=0.9852746725082397
Bounding box: startX=193, startY=121, endX=267, endY=225
Face shape: (104, 74, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Detection 0: confidence=0.9902476668357849
Bounding box: startX=193, startY=121, endX=266, endY=224
Face shape: (103, 73, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Detection 0: confidence=0.9899994730949402
Bounding box: startX=193, startY=121, endX=266, endY=224
Face shape: (103, 73, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Detection 0: confidence=0.9855647683143616
Bounding box: startX=193, startY=121, endX=266, endY=224
Face shape: (103, 73, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Detection 0: confidence=0.9920416474342346
Bounding box: startX=191, startY=121, endX=266, endY=222
Face shape: (101, 75, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Detection 0: confidenc

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.9005066752433777
Bounding box: startX=171, startY=62, endX=272, endY=191
Face shape: (129, 101, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Detection 0: confidence=0.9813620448112488
Bounding box: startX=171, startY=53, endX=272, endY=194
Face shape: (141, 101, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Detection 0: confidence=0.9948467016220093
Bounding box: startX=170, startY=41, endX=267, endY=185
Face shape: (144, 97, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Detection 0: confidence=0.9984214305877686
Bounding box: startX=169, startY=37, endX=268, endY=176
Face shape: (139, 99, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.9993295669555664
Bounding box: startX=169, startY=40, endX=271, endY=179
Face shape: (139, 102, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Detection 0: confidence=0.9996894598007202
Bounding box: startX=170, startY=43, endX=272, endY=184
Face shape: (141, 102, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.9869497418403625
Bounding box: startX=179, startY=60, endX=276, endY=193
Face shape: (133, 97, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Detection 0: confidence=0.9521892070770264
Bounding box: startX=179, startY=59, endX=276, endY=193
Face shape: (134, 97, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Detection 0: confidence=0.9438300728797913
Bounding box: startX=178, startY=60, endX=274, endY=192
Face shape: (132, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.9264489412307739
Bounding box: startX=180, startY=59, endX=275, endY=190
Face shape: (131, 95, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Detection 0: confidence=0.9165816307067871
Bounding box: startX=179, startY=58, endX=274, endY=183
Face shape: (125, 95, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Detection 0: confidence=0.8424860239028931
Bounding box: startX=176, startY=58, endX=272, endY=184
Face shape: (126, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Detection 0: confidence=0.9345906376838684
Bounding box: startX=187, startY=118, endX=263, endY=222
Face shape: (104, 76, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Detection 0: confidence=0.9816122651100159
Bounding box: startX=187, startY=118, endX=263, endY=221
Face shape: (103, 76, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Detection 0: confidence=0.9328957796096802
Bounding box: startX=186, startY=117, endX=265, endY=223
Face shape: (106, 79, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Detection 0: confidence=0.9635069966316223
Bounding box: startX=188, startY=120, endX=263, endY=222
Face shape: (102, 75, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Detection 0: confidence=0.9485028982162476
Bounding box: startX=188, startY=120, endX=264, endY=224
Face shape: (104, 76, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.9211829304695129
Bounding box: startX=188, startY=121, endX=265, endY=226
Face shape: (105, 77, 3)
1/1 ━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Detection 0: confidence=0.9588354825973511
Bounding box: startX=193, startY=124, endX=263, endY=229
Face shape: (105, 70, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Detection 0: confidence=0.9181535243988037
Bounding box: startX=193, startY=124, endX=263, endY=229
Face shape: (105, 70, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Detection 0: confidence=0.9254074096679688
Bounding box: startX=193, startY=123, endX=263, endY=230
Face shape: (107, 70, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Detection 0: confidence=0.9080867767333984
Bounding box: startX=193, startY=124, endX=263, endY=230
Face shape: (106, 70, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Detection 0: confidence=0.9361456036567688
Bounding box: startX=193, startY=123, endX=266, endY=229
Face shape: (106, 73, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Detection 0: confidence=0.8864424824714661
Bounding box: startX=193, startY=123, endX=266, endY=230
Face shape: (107, 73, 3)
1/1 ━━━━━━━━━━━━━━━━━━

AttributeError: 'WebcamVideoStream' object has no attribute 'release'